In [1]:
from firedrake import *
from irksome import RadauIIA, Dt, MeshConstant, TimeStepper, PEPRK, GaussLegendre, ContinuousPetrovGalerkinScheme
from scipy.io import loadmat
from scipy.interpolate import RegularGridInterpolator

firedrake:WARNING OMP_NUM_THREADS is not set or is set to a value greater than 1, we suggest setting OMP_NUM_THREADS=1 to improve performance


In [22]:
from firedrake import *
from irksome import RadauIIA, Dt, MeshConstant, TimeStepper, PEPRK, GaussLegendre, ContinuousPetrovGalerkinScheme
from scipy.io import loadmat
from scipy.interpolate import RegularGridInterpolator

butcher_tableau = ContinuousPetrovGalerkinScheme(2)

Nx = 20
Ny = 20
L = 0.5
h = 1
msh_p = PeriodicRectangleMesh(Nx, Ny, L, h, direction='x')#, quadrilateral=True)
V = VectorFunctionSpace(msh_p, "CG", 2, dim=2)
W = FunctionSpace(msh_p, "CG", 1)
Z = V*W

MC = MeshConstant(msh_p)
# dt = 0.002
dt = MC.Constant(0.1)
Re = MC.Constant(90.0)

In [23]:
# Poiseuille laminar flow in [0, 2]
u0 = Function(V)
x, y = SpatialCoordinate(msh_p)
psi = sin(pi*y)

u0.assign(project(as_vector([psi, 0]), V))
p0 = Function(W)

In [24]:
t = MC.Constant(0.0)
# Initialize solution function
up = Function(Z)
u_, p_ = up.subfunctions
u_.rename('Velocity')
p_.rename("Pressure")

# Assign initial conditions
u_.assign(u0)
p_.assign(p0)

# Set up variational problem
dx_p = dx(msh_p)

u, p = split(up)
v, w = TestFunctions(Z)
F = (inner(Dt(u), v) * dx_p + inner(dot(grad(u), u), v) * dx_p
     + 1/Re * inner(grad(u), grad(v)) * dx_p - inner(p, div(v)) * dx_p
     + inner(div(u), w) * dx_p)

bcs = [DirichletBC(Z.sub(0), as_vector([0, 0]), (1, 2))]
u_exact = Function(V)
psi = sin(pi*y)*exp(-pi*pi*t/Re)
u_exact.assign(project(as_vector([psi, 0]), V))
error = u_ - u_exact
E = inner(error, error)*dx_p

# Solver parameters
luparams = {"mat_type": "aij",
            "snes_type": "newtonls",
            "snes_monitor": None,
            "ksp_type": "preonly",
            "pc_type": "lu",
            "pc_factor_mat_solver_type": "mumps",
            "snes_linesearch_type": "l2",
            "snes_force_iteration": 1,
            "snes_rtol": 1e-8,
            "snes_atol": 1e-8
            }

# Define nullspace for pressure
nsp = MixedVectorSpaceBasis(Z, [Z.sub(0), VectorSpaceBasis(constant=True, comm=msh_p.comm)])

stepper = TimeStepper(F, butcher_tableau, t, dt, up, bcs=bcs,
                      solver_parameters=luparams, nullspace=nsp)

errors = []
times = [float(t)]
# out = File("ns_Poiseuille_Re300.pvd")
# out.write(u_, p_, time=float(t))


et = assemble(E)

print(f"  Error = {et}")
errors.append(et)

# Time stepping loop
for _ in range(100):
     print(f"Stepping from time {float(t)}")
     stepper.advance()
     t.assign(float(t) + float(dt))
     u_exact = Function(V)
     psi = sin(pi*y)*exp(-pi*pi*t/Re)
     u_exact.assign(project(as_vector([psi, 0]), V))
     error = u_ - u_exact
     E = inner(error, error)*dx_p
     et = assemble(E)
     print(f"  Error = {et}")
     errors.append(et)
     times.append(float(t))

  Error = 9.859221111133075e-34
Stepping from time 0.0
  0 SNES Function norm 7.900124817919e-05
  1 SNES Function norm 2.336554013141e-11
  Error = 2.112535641277595e-11
Stepping from time 0.1
  0 SNES Function norm 9.433664422289e-05
  1 SNES Function norm 1.560212230587e-12
  Error = 1.3014801873347074e-11
Stepping from time 0.2
  0 SNES Function norm 9.330678313273e-05
  1 SNES Function norm 7.622681734317e-13
  Error = 1.2211563533728543e-11
Stepping from time 0.30000000000000004
  0 SNES Function norm 9.228848978182e-05
  1 SNES Function norm 4.969209954790e-13
  Error = 1.188047172052599e-11
Stepping from time 0.4
  0 SNES Function norm 9.128152569166e-05
  1 SNES Function norm 3.987932147155e-13
  Error = 1.1625514845847575e-11
Stepping from time 0.5
  0 SNES Function norm 9.028568665225e-05
  1 SNES Function norm 3.388049151455e-13
  Error = 1.1385257264463206e-11
Stepping from time 0.6
  0 SNES Function norm 8.930080075143e-05
  1 SNES Function norm 2.944697021222e-13
  Error

In [25]:
np.savez('errors_N20', errors=errors, times=times)

In [2]:
error_N = []

data = np.load('errors_N3.npz')
errors = data['errors']
error_N.append(np.sum(errors)/len(errors))

data = np.load('errors_N10.npz')
errors = data['errors']
error_N.append(np.sum(errors)/len(errors))

data = np.load('errors_N20.npz')
errors = data['errors']
error_N.append(np.sum(errors)/len(errors))

data = np.load('errors_N50.npz')
errors = data['errors']
error_N.append(np.sum(errors)/len(errors))

In [4]:
np.savez('errors', error_N=error_N, N=[3,10,20,50])

In [ ]:

plt.loglog([3, 10, 20, 50], error_N, '-o')

NameError: name 'plt' is not defined

In [30]:
# slope value
np.polyfit(np.log([1/3, 1/10, 1/20, 1/50]), np.log(error_N), 1)


array([ 6.87080976, -5.39747153])